In [29]:
import numpy as np
import xgboost as xgb
import re


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [267]:
# Train an xgboost model, get the booster, and dump the trees.
# Write a function that eats a tree and spits out a function
# that generates the output of the tree.

Ytrain = np.argmax(mnist.train.labels, axis=1)
Ytest = np.argmax(mnist.test.labels, axis=1)

xgc = xgb.XGBClassifier(n_estimators=100, objective='multi:softmax')

In [249]:
l = list(zip(mnist.test.images[:1000,:], Ytest[:1000]))

7

In [268]:
model = xgc.fit(mnist.train.images[:1000,:], Ytrain[:1000])

In [269]:
booster = model.booster()

In [274]:
trees = booster.get_dump()

In [282]:
trees[0]

'0:[f435<0.00196078] yes=1,no=2,missing=1\n\t1:[f455<0.072549] yes=3,no=4,missing=3\n\t\t3:[f457<0.778431] yes=7,no=8,missing=7\n\t\t\t7:leaf=-0.0386082\n\t\t\t8:leaf=0.137097\n\t\t4:[f379<0.439216] yes=9,no=10,missing=9\n\t\t\t9:leaf=0.410965\n\t\t\t10:leaf=-0.0154639\n\t2:[f157<0.998039] yes=5,no=6,missing=5\n\t\t5:leaf=-0.0551325\n\t\t6:[f153<0.0196078] yes=11,no=12,missing=11\n\t\t\t11:leaf=0.0192308\n\t\t\t12:leaf=-0.0327869\n'

In [284]:
re.findall('[0-9*]+:+.*no=[0-9]', trees[0])

['0:[f435<0.00196078] yes=1,no=2',
 '1:[f455<0.072549] yes=3,no=4',
 '3:[f457<0.778431] yes=7,no=8',
 '4:[f379<0.439216] yes=9,no=1',
 '2:[f157<0.998039] yes=5,no=6',
 '6:[f153<0.0196078] yes=11,no=1']

In [290]:
re.findall('[0-9*]+:+.*no=[0-9]', trees[102])

['0:[f543<0.0980392] yes=1,no=2',
 '1:[f513<0.0647059] yes=3,no=4',
 '3:[f468<0.994118] yes=7,no=8',
 '4:[f319<0.00784314] yes=9,no=1',
 '2:[f345<0.0764706] yes=5,no=6',
 '5:[f180<0.0980392] yes=11,no=1',
 '6:[f638<0.0745098] yes=13,no=1']

In [31]:
re.findall('\[(.*?)\]', trees[0])

['f437<0.00196078', 'f577<0.00980392', 'f319<0.0137255', 'f436<0.798039']

In [42]:
leafs = re.findall('[0-9*]+:+leaf.*', trees[0])

['5:leaf=0.118699',
 '6:leaf=-0.168627',
 '7:leaf=-0.191257',
 '8:leaf=-0',
 '2:leaf=-0.195115']

In [270]:
class xgboost_tree_evaluator():

    '''
    How to use this class:
    
    import xgboost
    import xgboost_tree_evaluator
    
    # Prep data
    
    xgb = xgboost.XGBClassifier(params)
    model = xgb.fit(X, y)
    
    evaluator = xgboost_tree_evaluator()
    
    tree_outputs = evaluator.evaluate_trees(model, data)
    
    NOTE:  X must be a numpy array!  Pandas DataFrames would need more work.
    
    On my machine I was able to evaluate mnist.train.images on 100 trees in
    less than a second.  This should hopefully be fast enough.
    '''
    
    def __init__(self):
        self.tree_dicts = []
        self.outputs = []
    
    def create_tree(self, tree_str):

        decisions = re.findall('[0-9*]+:+.*no=+[0-9]*', tree_str)
        leafs = re.findall('[0-9*]+:+leaf.*', tree_str)

        tree = {}
        for decision in decisions:
            k = int(decision.rsplit(':')[0])

            ineq = re.findall('\[(.*?)\]', decision)[0].split('<')
            p = int(ineq[0][1:])
            v = float(ineq[1])

            yes = int(re.findall('yes=+[0-9]*', decision)[0].split('=')[1])
            no = int(re.findall('no=+[0-9]*', decision)[0].split('=')[1])

            tree[k] = (p, v, yes, no)

        for leaf in leafs:

            k = int(leaf.rsplit(':')[0])
            v = float(leaf.split('=')[1])

            tree[k] = v

        return tree

    def eval_tree(self, tree, images):

        i = 0

        curr = tree[i]
        outputs = []

        for image in images:
            #count = 0
            while type(curr) == tuple:# and count < 10:
                #print(curr)
                if image[curr[0]] < curr[1]:
                    curr = tree[curr[2]]
                else:
                    curr = tree[curr[3]]
                #count+=1
            outputs.append(curr)

        return outputs

    def evaluate_trees(self, model, data):
        
        booster = model.booster()
        trees = booster.get_dump()
        
        for i, tree in enumerate(trees):
            print(i)
            curr_tree = self.create_tree(tree)
            outputs = self.eval_tree(curr_tree, data)
            
            self.tree_dicts.append(curr_tree)
            self.outputs.append(outputs)
            
        return np.array(self.outputs)    

In [271]:
evaluator = xgboost_tree_evaluator()

In [272]:
outputs = evaluator.evaluate_trees(model, mnist.train.images)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [273]:
outputs.shape

(1000, 55000)

In [195]:
trees[64]

'0:[f403<0.213726] yes=1,no=2,missing=1\n\t1:[f235<0.00392157] yes=3,no=4,missing=3\n\t\t3:[f295<0.34902] yes=7,no=8,missing=7\n\t\t\t7:leaf=-0.0146112\n\t\t\t8:leaf=0.0590944\n\t\t4:[f270<0.0254902] yes=9,no=10,missing=9\n\t\t\t9:leaf=0.0170657\n\t\t\t10:leaf=-0.0681045\n\t2:[f322<0.990196] yes=5,no=6,missing=5\n\t\t5:leaf=-0.07278\n\t\t6:leaf=-0.0026538\n'

In [203]:
t_64 = evaluator.create_tree(trees[64])

In [204]:
evaluator.eval_tree(t_64, [mnist.train.images[0]])

(403, 0.213726, 1, 2)
(235, 0.00392157, 3, 4)
(270, 0.0254902, 9, 10)


[-0.0681045]

In [205]:
t_64

{0: (403, 0.213726, 1, 2),
 1: (235, 0.00392157, 3, 4),
 2: (322, 0.990196, 5, 6),
 3: (295, 0.34902, 7, 8),
 4: (270, 0.0254902, 9, 10),
 5: -0.07278,
 6: -0.0026538,
 7: -0.0146112,
 8: 0.0590944,
 9: 0.0170657,
 10: -0.0681045}

In [187]:
mnist.train.images[0][270]

0.74117649

In [189]:
print(trees[64])

0:[f403<0.213726] yes=1,no=2,missing=1
	1:[f235<0.00392157] yes=3,no=4,missing=3
		3:[f295<0.34902] yes=7,no=8,missing=7
			7:leaf=-0.0146112
			8:leaf=0.0590944
		4:[f270<0.0254902] yes=9,no=10,missing=9
			9:leaf=0.0170657
			10:leaf=-0.0681045
	2:[f322<0.990196] yes=5,no=6,missing=5
		5:leaf=-0.07278
		6:leaf=-0.0026538



In [155]:
test[437] = 0
test[437]

0.0

In [156]:
eval_tree(t, [test])

[-0.191257]